# Testing

In [6]:
import openai
import pandas as pd
from random import randrange
from math import exp
from dotenv import load_dotenv

# 개인 api key
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

m_num = 3
"""
- `curie:ft-personal-2023-05-31 08-28-50` : csi 데이터 학습
- `curie:ft-personal-2023-06-11 18-10-30` : 總 데이터 사고경위만 학습
- `curie:ft-personal-2023-06-12-06-34-37` : 總 데이터 키워드化하여 사고경위와 함께 학습
"""
ft_models = {1 : "curie:ft-personal-2023-05-31 08-28-50",
            2 : "curie:ft-personal-2023-06-11 18-10-30",
            3 : "curie:ft-personal-2023-06-12-06-34-37"}
pp_datas = {1 : "C:/Users/bhyoo/Desktop/Causual Analysis/CSI_DESKTOP_Test/s_filtered_sieben_codex_v3_prepared.jsonl",
            2 : "C:/Users/bhyoo/Desktop/Causual Analysis/GPT Fine-tuning/prepocess/prepocessed.jsonl",
            3 : "C:/Users/bhyoo/Desktop/Causual Analysis/GPT Fine-tuning/prepocess/prepocessed with keywords.jsonl"}
ft_model = ft_models[m_num]
pp_data = pp_datas[m_num]

## Data Load

In [30]:
no = randrange(1, 1_001)
# no = 0

sample_data = pd.read_json(pp_data, lines=True).iloc[no]
sample_prompt = sample_data['prompt'] + "\n\n###\n\n"
sample_completion = sample_data['completion']

print(f"[row number] {no}\n[prompt] {sample_prompt}\n[completion] {sample_completion}")

[row number] 979
[prompt] nan#nan#nan#건축 > 가설공사#nan#nan#nan#nan#60~69%#여름#새벽#모재 절단을 위해 전동 원형 톱을 이용중 안전커버가 닫기지 않은 상태에서 톱을 바닥에 놓으므로 톱이 튀어서 톱날에 오른쪽 다리 장단지 부위에 약 20cm정도의 자상이 발생한 사고임

###


[completion] 절단, 베임, 찔림


## Prompt

In [31]:
"""Prompt 작성"""
# sample_prompt = '''개인 지병에 의한 병사이며2019.07.22. 월요일 오전 11시 30분경 설비 보온재 시공 근로자 OOO씨가 106동 지하1층 주차장에서이동식 틀비계를 끌어안고 있는 것을 동료 근로자 OOO씨가 발견하고 “왜그래? 왜그래?”라고 물으니OOO씨가 “기운이 없어”라고 답하며 주저 앉으려고 하자 조한월씨가 부축하여 바닥에 누인 다음119로 부천 카톨릭 성모병원으로 이송하였고부천 카톨릭 성모병원에서 뇌출혈 진단을 받고 2019.07.22 당일 수술 후 중환자실에 입원중2019.08.04 오전 07시 36분에 병사한 것임.날씨 맑음 기온 31℃ 습도 74%'''
# 정답 : 기타

'Prompt 작성'

## Completion

In [32]:
"""답만 보는 코드"""
res = openai.Completion.create(model=ft_model, prompt=sample_prompt, max_tokens=50, temperature=0.2, logprobs=8)
res['choices'][0]['text'].lstrip(' ').split(' ')[0]

'절단,'

In [33]:
res['choices'][0]['logprobs']['top_logprobs'][0]

<OpenAIObject at 0x1bc978939f0> JSON: {
  " ": -6.1572285,
  "bytes: \\xeb": -7.2011843,
  "bytes: \\xec": -0.0028715432,
  "bytes:\\xec": -13.375288,
  "bytes:\\xec\\x9d": -13.66861
}

In [34]:
"""JSON"""
res

<OpenAIObject text_completion id=cmpl-7QYZkHD93xR2npUTjm4BfEtrHV6ty at 0x1bc93421090> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": {
        "text_offset": [
          158,
          158,
          158,
          160,
          160,
          161,
          162,
          162,
          162,
          164,
          164,
          164,
          165,
          166,
          166,
          166,
          168,
          168,
          168,
          169,
          170,
          170,
          170,
          172,
          172,
          172,
          173,
          173,
          175,
          175,
          175,
          176,
          177,
          177,
          177,
          179,
          179,
          179,
          180,
          180,
          182,
          182,
          182,
          183,
          183,
          183,
          185,
          185,
          185,
          186
        ],
        "token_logprobs": [
 

In [36]:
! openai -k "{OPENAI_API_KEY}" api completions.create -m "{ft_model}" -p "{sample_prompt}"

nan#nan#nan#건축 > 가설공사#nan#nan#nan#nan#60~69%#여름#새벽#모재 절단을 위해 전동 원형 톱을 이용중 안전커버가 닫기지 않은 상태에서 톱을 바닥에 놓으므로 톱이 튀어서 톱날에 오른쪽 다리 장단지 부위에 약 20cm정도의 자상이 발생한 사고임

###

 절단, 베임


In [44]:
neo_target = list(res['choices'][0]['logprobs']['top_logprobs'][0])
"""
<OpenAIObject at 0x1bc978939f0> JSON: {
    " ": -6.1572285,
    "bytes: \\xeb": -7.2011843,
    "bytes: \\xec": -0.0028715432,
    "bytes:\\xec": -13.375288,
    "bytes:\\xec\\x9d": -13.66861
}
"""
print(neo_target)
# neo_target.to_dict()
for n, w in neo_target:
    print(n, w)

['bytes:\\xec', ' ', 'bytes: \\xec', 'bytes: \\xeb', 'bytes:\\xec\\x9d']


ValueError: too many values to unpack (expected 2)

In [54]:
from math import exp

print(b"\\xeb".decode('utf-8'), exp(-0.0028715432))

\xeb 0.9971325757366633
